# Tutorial for Mathematical Morphology

This file is meant to demonstrate the use of gstlearn for Morphological Operations

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

Setting some global variables

In [ ]:
# Set the Global Options
verbose = True
flagGraphic = True

# Define the Space Dimension
ndim = 2
gl.ASpaceObject_defineDefaultSpace(gl.ESpaceType.SPACE_RN, ndim)

# Set the Seed for the Random Number generator
gl.law_set_random_seed(5584)

In [ ]:
nx = ny = 100
nxy = [nx,ny]
ntotal = nx * ny
dx = dy = 0.01

In several usages, we will need a VectorDouble dimensionned to the total number of pixels. This is created next.

In [ ]:
localVD = gl.VectorDouble(ntotal)

Generating an initial square grid covering a 1 by 1 surface (100 meshes along each direction).

In [ ]:
grid = gl.DbGrid.create(nxy, [dx,dy])
model = gl.Model_createFromParam(gl.ECov.CUBIC, 0.1, 1.)
gl.simtub(None, grid, model)
if verbose:
    grid.display()
if flagGraphic:
    ax = gp.grid(grid,title="Initial Data Set")

## Basic operations

We retreive the newly simulated variable (called *Simu*) from the *grid* Db into a local Vector (called *tab*). This vector is then transformed by thresholding and loaded into an image object (called *image*). This object is very efficient as each pixel is stored into a single *bit*. A second Image object (called *image2*) is created and will be used in subsequent diadic operations.

In [ ]:
vmin = -1
vmax = +1
image = gl.morpho_image_manage(nxy)
image2 = gl.morpho_image_manage(nxy)
tab = grid.getColumn("Simu")
image = gl.morpho_double2image(nxy,tab,vmin,vmax)

In [ ]:
volume = gl.morpho_count([nx,ny], image)
print("Grain Volume =",volume, " /",ntotal,"(pixels)\n")

For visualization (and i the current version), we must first convert the image into a vector and load it into a grid.

In [ ]:
if flagGraphic:
    gl.morpho_image2double(nxy, image, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Initial Image",gl.ELoc.Z)
    ax = gp.grid(grid)

The next step interchanges grain and pore

In [ ]:
gl.morpho_negation(nxy, image, image2)

In [ ]:
if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Negative Image",gl.ELoc.Z)
    ax = gp.grid(grid)

## Basic Mophological Image transformations

We start with the initial image and perform an erosion. The second argument defines the type of structuring element: either Cross (0) or Block (1)

In [ ]:
gl.morpho_erosion(nxy, 0, [1,1], image, image2)

if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Erosion - Cross",gl.ELoc.Z)
    ax = gp.grid(grid)

We check the result of ersosion when choosing the Block structuring element

In [ ]:
gl.morpho_erosion(nxy, 1, [1,1], image, image2)

if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Erosion - Block",gl.ELoc.Z)
    ax = gp.grid(grid)

We now perform the dilation of the Initial image (only the Cross structuring element will be used in the next paragraphs)

In [ ]:
gl.morpho_dilation(nxy, 0, [1,1], image, image2)

if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Dilation - Cross",gl.ELoc.Z)
    ax = gp.grid(grid)

Combining the elementary operations (Erosion and Dilation), we can perform directly an opening

In [ ]:
gl.morpho_opening(nxy, 0, [1,1], image, image2)

if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Opening - Cross",gl.ELoc.Z)
    ax = gp.grid(grid)

And the closing

In [ ]:
gl.morpho_closing(nxy, 0, [1,1], image, image2)

if flagGraphic:
    gl.morpho_image2double(nxy, image2, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Closing - Cross",gl.ELoc.Z)
    ax = gp.grid(grid)

## Connected components

Starting from the Initial image, we now wish to determine the connected components

In [ ]:
gl.morpho_negation(nxy, image, image)

if flagGraphic:
    gl.morpho_image2double(nxy, image, 0, 1., 0., localVD)
    iuid = grid.addColumns(localVD,"Staring For CC",gl.ELoc.Z)
    ax = gp.grid(grid)

In [ ]:
ncomp = gl.morpho_labelling(nxy, 0, 0, image, gl.TEST, localVD)
print("Number of connected components =", ncomp,"\n")

if flagGraphic:
    iuid = grid.addColumns(localVD,"Connect Components by Rank",gl.ELoc.Z)
    ax = gp.grid(grid)

In [ ]:
ncomp = gl.morpho_labelling(nxy, 0, 1, image, gl.TEST, localVD)
print("Number of connected components =", ncomp,"\n")

if flagGraphic:
    iuid = grid.addColumns(localVD,"Connect Components by Volume",gl.ELoc.Z)
    ax = gp.grid(grid)